<a href="https://colab.research.google.com/github/sayakpaul/TF-2.0-Hacks/blob/master/data2vec_vision_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we'll fine-tune a model pre-trained using data2vec. We'll leverage the `transformers` library for loading the pre-trained model and we'll then append a custom classification head for fine-tuning. For data handling we'll use `tf.data`.

Checkout the data2vec paper [here](https://arxiv.org/abs/2202.03555) and the model documentation page [here](https://huggingface.co/docs/transformers/main/en/model_doc/data2vec).

## Setup

In [ ]:
# Otherwise TF's data2vec vision model won't be available.
# Date: May 06, 2022
!pip install git+https://github.com/huggingface/transformers -q

## Imports

In [ ]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers

from transformers import TFAutoModelForImageClassification, TFData2VecVisionModel
from transformers import create_optimizer

import tensorflow_datasets as tfds

tfds.disable_progress_bar()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Primary constants

In [ ]:
# Model
IMAGE_SIZE = [224, 224]
MODEL_ID = "facebook/data2vec-vision-base"  # pre-trained model from which to fine-tune

# Data
BATCH_SIZE = 16  # on Colab/GPU, a higher batch size may throw(OOM)

# Dataset
CLASSES = [
    "dandelion",
    "daisy",
    "tulips",
    "sunflowers",
    "roses",
]  # don't change the order

# Other constants
MEAN = tf.constant([0.485 * 255, 0.456 * 255, 0.406 * 255])  # imagenet mean
STD = tf.constant([0.229 * 255, 0.224 * 255, 0.225 * 255])  # imagenet std
AUTO = tf.data.AUTOTUNE

## Data related utilities

In [ ]:
# Closely aligns with https://github.com/huggingface/transformers/blob/main/src/transformers/models/beit/feature_extraction_beit.py
# in regards to resizing and normalization.


def make_dataset(dataset: tf.data.Dataset, train: bool, image_size: int = IMAGE_SIZE):
    def preprocess(image, label):
        # for training, do augmentation
        if train:
            if tf.random.uniform(shape=[]) > 0.5:
                image = tf.image.flip_left_right(image)
        image = tf.image.resize(image, size=image_size, method="bicubic")
        image = (image - MEAN) / STD  # normalization
        return image, label

    if train:
        dataset = dataset.shuffle(BATCH_SIZE * 10)

    dataset = dataset.map(preprocess, AUTO).batch(BATCH_SIZE)
    # Transpose because the `transformers` model has a leading channel dimension.
    dataset = dataset.map(lambda x, y: (tf.transpose(x, [0, 3, 1, 2]), y), AUTO)
    return dataset.prefetch(AUTO)

## Load and visualize the `tf_flowers` dataset

In [ ]:
train_dataset, val_dataset = tfds.load(
    "tf_flowers",
    split=["train[:90%]", "train[90%:]"],
    as_supervised=True,
    try_gcs=False,  # gcs_path is necessary for tpu,
)

num_train = tf.data.experimental.cardinality(train_dataset)
num_val = tf.data.experimental.cardinality(val_dataset)
print(f"Number of training examples: {num_train}")
print(f"Number of validation examples: {num_val}")

In [ ]:
train_dataset = make_dataset(train_dataset, True)
val_dataset = make_dataset(val_dataset, False)

In [ ]:
sample_images, sample_labels = next(iter(train_dataset))

plt.figure(figsize=(5 * 3, 3 * 3))
for n in range(15):
    ax = plt.subplot(3, 5, n + 1)
    image = (sample_images[n].numpy().transpose(1, 2, 0) * STD + MEAN).numpy()
    image = (image - image.min()) / (
        image.max() - image.min()
    )  # convert to [0, 1] for avoiding matplotlib warning
    plt.imshow(image)
    plt.title(CLASSES[sample_labels[n]])
    plt.axis("off")
plt.tight_layout()
plt.show()

## Model building utilities

In [ ]:
def get_model():
    base_model = TFData2VecVisionModel.from_pretrained(MODEL_ID)
    base_model.trainable = True

    inputs = layers.Input((3, IMAGE_SIZE[0], IMAGE_SIZE[1]))
    x = base_model(inputs, training=False)
    outputs = layers.Dense(5)(x.last_hidden_state[:, 0])
    model = keras.Model(inputs, outputs)

    return model

In [ ]:
model = get_model()
model.summary()

## Compile model and train it

In [ ]:
num_train_epochs = 3
learning_rate = 3e-5
weight_decay_rate = 0.01
num_warmup_steps = 0


num_train_steps = (num_train // BATCH_SIZE) * num_train_epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=learning_rate,
    num_train_steps=num_train_steps,
    weight_decay_rate=weight_decay_rate,
    num_warmup_steps=num_warmup_steps,
)

In [ ]:
model.compile(
    optimizer=optimizer,
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=num_train_epochs)

## Visualize training progress

In [ ]:
result = pd.DataFrame(history.history)
fig, ax = plt.subplots(2, 1, figsize=(10, 10))
result[["accuracy", "val_accuracy"]].plot(xlabel="epoch", ylabel="score", ax=ax[0])
result[["loss", "val_loss"]].plot(xlabel="epoch", ylabel="score", ax=ax[1])

## References

* https://github.com/sayakpaul/cait-tf/blob/main/notebooks/finetune.ipynb
* https://colab.research.google.com/drive/1M4CEl6Jgf4KgqLHb4_bi3acJvNNLZqZd